# MyMethods.jl -- Examples

This notebook illustrates basic functionalities of the MyMethods.jl package.

### Installation and Removal of MyMethods.jl

In [3]:
# Install the package from GitHub (first time only) 
#using Pkg; 
#Pkg.add(url="https://github.com/thomaswiemann/MyMethods.jl");

# To uninstall the package, simply use
#Pkg.rm("MyMethods")

   Updating git-repo `https://github.com/thomaswiemann/MyMethods.jl`


   Updating registry at `C:\Users\thomas\.julia\registries\General`


LoadError: package `MyMethods [18de7e97]` has same name or UUID as the active project

In [4]:
# Load MyMethods.jl into your Julia session 
using MyMethods

┌ Info: Precompiling MyMethods [18de7e97-ce2a-4ea9-8d3b-e4b0c3b0b515]
└ @ Base loading.jl:1278
┌ Warning: Package MyMethods does not have DataFrames in its dependencies:
│ - If you have MyMethods checked out for development and have
│   added DataFrames as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MyMethods
└ Loading DataFrames into MyMethods from project dependency, future warnings for MyMethods are suppressed.


### Exemplary Data  

Generate basic data to be used in the examples.

In [5]:
# Data parameters
N = 10000 # sample size
beta = [1,2,-1,4] # regression coefficients

# Data
_r = rand(N) # running variable
X = hcat(_r.*10, rand(N,3))# constant has mean 5
y = X*beta + randn(N,1); # outcome simulation

### myLS
myLS() generates a (weighted) least squares object.

In [9]:
# Estimate least squares fit
fit_LS = myLS(y, X);

In [11]:
# Calculate and print standard errors
inference(fit_LS)

LoadError: MethodError: no method matching getindex(::Nothing, ::Int64)